In [1]:
import pandas as pd
import numpy as np
import os
import io

In [20]:
all_files = os.listdir('Results')
metadata = pd.read_csv('ChIA-PET data/metadata.tsv', sep='\t')
# files_to_keep = list(metadata[~(metadata['Biosample term name'].str.contains('positive')) & ~(metadata['Biosample term name'].str.contains('activated')) & ~(metadata['Biosample term name'].str.contains('T-cell'))]['File accession'])
# biosamples = list(metadata[~(metadata['Biosample term name'].str.contains('positive')) & ~(metadata['Biosample term name'].str.contains('activated')) & ~(metadata['Biosample term name'].str.contains('T-cell'))]['Biosample term name'].unique())
df = pd.read_csv('../data/final_dataset.csv')

In [10]:
ctcf_files = list(metadata[~(metadata['Biosample term name'].str.contains('positive')) & ~(metadata['Biosample term name'].str.contains('activated')) & ~(metadata['Biosample term name'].str.contains('T-cell')) & (metadata['Experiment target'].str.contains('CTCF'))]['File accession'])
polr_files = list(metadata[~(metadata['Biosample term name'].str.contains('positive')) & ~(metadata['Biosample term name'].str.contains('activated')) & ~(metadata['Biosample term name'].str.contains('T-cell')) & (metadata['Experiment target'].str.contains('POLR2A'))]['File accession'])

sum_ctcf = 0
for i in ctcf_files:
    f = pd.read_csv('ChIA-PET data/' + i + '.bedpe', sep = '\t',  header = None)
    sum_ctcf = sum_ctcf + len(f)

sum_pol2r = 0
for i in polr_files:
    f = pd.read_csv('ChIA-PET data/' + i + '.bedpe', sep = '\t',  header = None)
    sum_pol2r = sum_pol2r + len(f)

In [21]:
print(sum_ctcf)
print(sum_pol2r)

39166005
15622208


In [ ]:
df['SYMBOL'].unique()

In [ ]:
all_files

In [19]:
df = pd.read_csv('Results/processed_ENCFF008OPR.csv')
df = df[['chr', 'start', 'end', 'pos_37', 'driver']]
df[['CTCF_interactions', 'CTCF_chains', 'POLR2A_interactions', 'POLR2A_chains']] = 0

for file in all_files:
    # if file.replace('processed_', '').replace('.csv', '') in files_to_keep:
    f = pd.read_csv('Results/' + file)
    df[['CTCF_interactions', 'CTCF_chains', 'POLR2A_interactions', 'POLR2A_chains']] = df[['CTCF_interactions', 'CTCF_chains', 'POLR2A_interactions', 'POLR2A_chains']].add(f[['CTCF_interactions', 'CTCF_chains', 'POLR2A_interactions', 'POLR2A_chains']])

df

,chr,start,end,pos_37,driver,CTCF_interactions,CTCF_chains,POLR2A_interactions,POLR2A_chains
0,1,43981392,43981392,44447064,0,509653,389533,441675,297371
1,1,193142068,193142068,193111198,1,1853064,1408350,494199,327505
2,1,239913497,239913497,240076797,0,1580027,1174006,199017,142874
3,1,189107161,189107161,189076292,0,229429,175222,6,4
4,1,212035724,212035724,212209066,1,3519256,2629966,1089722,728628
...,...,...,...,...,...,...,...,...,...
1043,X,54527681,54527681,54554114,0,110684,65841,11983,7685
1044,X,75622499,75622499,74842334,0,0,0,0,0
1045,X,50192749,50192749,49957400,0,145255,92787,1,1
1046,X,73034552,73034552,72254391,0,4,2,0,0


In [8]:
df.to_csv('final_interactions_result.csv', index = False)

In [16]:
df[df['pos_37'] == 339608]

,chr,start,end,pos_37,driver,CTCF_interactions,CTCF_chains,POLR2A_interactions,POLR2A_chains
335,16,289608,289608,339608,1,159802,124543,104219,69206


In [10]:
pd.read_csv('final_interactions_result.csv')

,chr,start,end,pos_37,driver,CTCF_interactions,CTCF_chains,POLR2A_interactions,POLR2A_chains
0,1,43981392,43981392,44447064,0,509653,389533,441675,297371
1,1,193142068,193142068,193111198,1,1549463,1169254,173754,107885
2,1,239913497,239913497,240076797,0,460811,328873,83415,59100
3,1,189107161,189107161,189076292,0,87071,70547,6,4
4,1,212035724,212035724,212209066,1,912001,670970,408752,278048
...,...,...,...,...,...,...,...,...,...
1043,X,54527681,54527681,54554114,0,3,3,66,39
1044,X,75622499,75622499,74842334,0,0,0,0,0
1045,X,50192749,50192749,49957400,0,2693,1882,1,1
1046,X,73034552,73034552,72254391,0,4,2,0,0
